In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import seaborn as sns
import rasterio as rio
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
#import Process_sb_tiff
import geotiff_processing.process_sb_tiff
import geotiff_processing.hillshades_pca
import geotiff_processing.geotiff_plotting
import geotiff_processing.write_mapfile

In [4]:
import datetime
def nowTime():
    return datetime.datetime.now().strftime("%H:%M:%S")
def now():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
def snow():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
def today():
    return datetime.datetime.now().strftime("%Y-%m-%d")
def stoday():
    return datetime.datetime.now().strftime("%Y%m%d")

In [5]:
tiff_dir = os.getcwd()

single_band_tiff = 'Anchieta_demo_dem.tif'

single_band_tiff = os.path.join(single_band_tiff.split('.')[0], single_band_tiff)
single_band_tiff

'Anchieta_demo_dem/Anchieta_demo_dem.tif'

In [6]:
single_band_tiff_path = os.path.join(tiff_dir, single_band_tiff)
single_band_tiff_path

'/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem.tif'

In [7]:
ph_colormap, ph_data_breaks, ph_percentile_breaks = geotiff_processing.process_sb_tiff.make_rgba_tiff_from_single_Band(single_band_tiff_path,
                                                                                                                       #data_min_max=datarange,
                                                                                                                       cmap_method='pseudo_hist_norm',
                                                                                                                       output_tif=True)

Wrote 1component LUT files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm_r.lut'
Wrote 1component LUT files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm_g.lut'
Wrote 1component LUT files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm_b.lut'
Wrote 1component LUT files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm_a.lut'
wrote 4component Lut files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_rgba_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm_qgis_color_table.txt''
wrote 4component Lut files to: '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_rgba_EMeraldCustomTerrain_0_to_54_pseudo_h

In [8]:
hs_azimuths, hs_altitudes = geotiff_processing.hillshades_pca.build_hs_az_al(start_al=30, al_inc=30, num_al_angles=2,
                                                                             start_az=90, num_az_angles=8)

azimuths = [0, 45, 90, 135, 180, 225, 270, 315]
altitudes = [30, 60]


In [9]:
hillshades, pcaComponents = geotiff_processing.hillshades_pca.build_hillshade(single_band_tiff_path=single_band_tiff_path,
                                                                              data_min_max=[min(ph_data_breaks), max(ph_data_breaks)],
                                                                              hs_azimuths=hs_azimuths,
                                                                              hs_altitudes=hs_altitudes,
                                                                              cmap=ph_colormap,
                                                                              process_pca=True)

New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az000_al30.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az000_al60.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az045_al30.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az045_al60.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az090_al30.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az090_al60.tif' generated successfully!
New RGBA geotiff '/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_hillshade_az13

In [10]:
writeLUTs=True
writeTifs=True
writeHillshades=True

mapfile_path = f"{single_band_tiff_path.split('.tif')[0]}"
if writeLUTs:
    mapfile_path = f"{mapfile_path}_LUT"
if writeTifs:
    mapfile_path = f"{mapfile_path}_AuxTif"
if writeHillshades:
    mapfile_path = f"{mapfile_path}_HS"


mapfile_path = f"{mapfile_path}.map"

mapfile_path

'/Users/bbloss/EMeraldCodes/ben_test_scratch/Anchieta_demo_dem/Anchieta_demo_dem_LUT_AuxTif_HS.map'

In [18]:
geotiff_processing.write_mapfile.generate_LUT_based_mapfile(single_band_tiff_path, mapfile_path, writeLUTs=True, writeTifs=True, writeHillshades=True)

## Mapfile written on 2024-04-16 at 15:18:54
MAP
    NAME "Anchieta_demo_dem"
    EXTENT 328021.1052 7701182.71 331633.0948 7703966.89
    SIZE 451 348
    PROJECTION
        "init=EPSG:31984"  # Set the EPSG code
    END

    WEB
        METADATA
          'wms_title'          'Anchieta_demo_dem'
          'wms_enable_request' '*'
          'wms_srs'            'EPSG:31984'
        END 
    END
    
    LAYER
        NAME "Anchieta_demo_dem"
        DATA "Anchieta_demo_dem.tif"
        TYPE RASTER
        STATUS ON
        EXTENT 328021.1052 7701182.71 331633.0948 7703966.89
        PROJECTION
            "init=EPSG:31984"
        END
        PROCESSING "NODATA=1.701410009187828e+38"  # Set your desired Nodata value
        PROCESSING "LUT=0.0:0, 60.0:255"  # Define the LUT (elevation:color)
    END
    
    LAYER
        NAME "Anchieta_demo_dem_EMeraldCustomTerrain_0_to_54_pseudo_hist_norm"
        DATA "Anchieta_demo_dem.tif"
        TYPE RASTER
        STATUS ON
        EXTENT 3280